# Automunge under automation

Automunge is available now for pip install:

In [1]:
# !pip install Automunge

Or to upgrade (we currently roll out upgrades pretty frequently):

In [2]:
# !pip install Automunge --upgrade

Once installed, run this in a local session to initialize:

In [3]:
from Automunge import Automunger
am = Automunger.AutoMunge()

Under automation, the automunge(.) function will: 
- normalize numeric features
- binarize bounded categoric features
- hash unbounded categoric features
- encode date-time entries

To demonstrate, let's encode the Titanic set, a well known benchmark:

In [4]:
import pandas as pd

#titanic set
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Here is what the data looks like in a raw form.

In [5]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We'll need to designate to automunge any columns that are to be treated as labels or ID sets.

In [6]:
#titanic set
labels_column = 'Survived'
trainID_column = 'PassengerId'

We can then pass these dataframes to the automunge(.) function for processing.

Note that the function call returns 17 sets. some of which may be empty based on parameter configurations. It's an unusual convention but we find that by having one return configuration for all scenarios it keeps things simple.

In [7]:
train, trainID, labels, \
validation1, validationID1, validationlabels1, \
validation2, validationID2, validationlabels2, \
test, testID, testlabels, \
labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
featureimportance, postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column)

_______________
Begin Automunge processing

evaluating column:  Pclass
processing column:  Pclass
    root category:  text
 returned columns:
['Pclass_1.0', 'Pclass_2.0', 'Pclass_3.0']

evaluating column:  Name
processing column:  Name
    root category:  hash
 returned columns:
['Name_hash_0', 'Name_hash_1', 'Name_hash_2', 'Name_hash_3', 'Name_hash_4', 'Name_hash_5', 'Name_hash_6', 'Name_hash_7', 'Name_hash_8', 'Name_hash_9', 'Name_hash_10', 'Name_hash_11', 'Name_hash_12', 'Name_hash_13']

evaluating column:  Sex
processing column:  Sex
    root category:  bnry
 returned columns:
['Sex_bnry']

evaluating column:  Age
processing column:  Age
    root category:  nmbr
 returned columns:
['Age_nmbr']

evaluating column:  SibSp
processing column:  SibSp
    root category:  nmbr
 returned columns:
['SibSp_nmbr']

evaluating column:  Parch
processing column:  Parch
    root category:  nmbr
 returned columns:
['Parch_nmbr']

evaluating column:  Ticket
processing column:  Ticket
    root categ

The returned data can be accessed in the sets:
train, trainID, labels

In [8]:
train.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Cabin_hash,Pclass_1.0,Pclass_2.0,Pclass_3.0,Name_hash_0,...,Name_hash_9,Name_hash_10,Name_hash_11,Name_hash_12,Name_hash_13,Ticket_hash_0,Ticket_hash_1,Ticket_hash_2,Embarked_1010_0,Embarked_1010_1
709,1,0.000000,0.432550,0.767199,-0.341261,175,0,0,1,124,...,0,0,0,0,0,541,0,0,0,0
439,1,0.100052,-0.474279,-0.473408,-0.436762,175,0,1,0,369,...,0,0,0,0,0,857,314,0,1,0
840,1,-0.745970,-0.474279,-0.473408,-0.488580,175,0,0,1,315,...,0,0,0,0,0,221,670,0,1,0
720,0,-1.822726,-0.474279,0.767199,0.016014,175,0,1,0,1015,...,0,0,0,0,0,920,0,0,1,0
39,0,-1.207437,0.432550,-0.473408,-0.421837,175,0,0,1,635,...,0,0,0,0,0,578,0,0,0,0


Note that the column headers of the returned data are different, now including suffix appenders logging the applied transformations.

Any carved out ID sets are included in the trainID set as well as an aggregated set of index numbers (since the function by default shuffles training data).

In [9]:
trainID.head()

,PassengerId,Automunge_index_797922035195
709,710,709
439,440,439
840,841,840
720,721,720
39,40,39


And labels:

In [10]:
labels.head()

,Survived_0.0,Survived_1.0
709,0,1
439,1,0
840,1,0
720,0,1
39,0,1


# a few more common parameters

A few options that might come up often:
- if we have test data available at same time as train data, we can also pass a test set
- if we want to carve out a validation set processed on the train set basis we can designate a ratio by the valpercent1 and/or valpercent2 parameter
- if we want to turn off printouts we can turn off with printstatus = False
- if we want to return numpy arrays instead of dataframes can pass pandasoutput = False
- for including with transformations a marker for entries that were subject to infill can pass NArw_marker = True
- for auto ML derived missing data infill can apply MLinfill = True

In [11]:
train, trainID, labels, \
validation1, validationID1, validationlabels1, \
validation2, validationID2, validationlabels2, \
test, testID, testlabels, \
labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
featureimportance, postprocess_dict \
= am.automunge(df_train,
               df_test = df_test,
               labels_column = labels_column,
               trainID_column = trainID_column,
               valpercent1 = 0.2, 
               printstatus = False, 
               pandasoutput = False,
               MLinfill = True,
               NArw_marker = True)

In [12]:
#the test data is returned in test, testID, testlabels
#here as a numpy array based on pandasoutput parameter
test

array([[ 1.        ,  0.37173158, -0.48163003, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        ,  1.3082061 ,  0.44518235, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  2.4319756 , -0.48163003, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.        ,  0.6714034 , -0.48163003, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        , -0.28454977, -0.48163003, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        , -1.9336214 ,  0.44518235, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

# Processing additional test data

Of the various returned sets, an important one is the final object we call the postprocess_dict. Think of this as a key to processing additioanl data on the original train set basis. If you intend to productionize a model we recomend saving externally such as with the pickle library. Once we have additional data we want to process we can pass it with the postprocess_dict to the postmunge(.) function.

In [13]:
test, testID, testlabels, \
labelsencoding_dict, postreports_dict \
= am.postmunge(postprocess_dict, df_test)

_______________
Begin Postmunge processing

______

processing column:  Pclass
    root category:  text

 returned columns:
['Pclass_NArw', 'Pclass_1.0', 'Pclass_2.0', 'Pclass_3.0']

______

processing column:  Name
    root category:  hash

 returned columns:
['Name_NArw', 'Name_hash_0', 'Name_hash_1', 'Name_hash_2', 'Name_hash_3', 'Name_hash_4', 'Name_hash_5', 'Name_hash_6', 'Name_hash_7', 'Name_hash_8', 'Name_hash_9', 'Name_hash_10', 'Name_hash_11', 'Name_hash_12', 'Name_hash_13']

______

processing column:  Sex
    root category:  bnry

 returned columns:
['Sex_bnry', 'Sex_NArw']

______

processing column:  Age
    root category:  nmbr

 returned columns:
['Age_nmbr', 'Age_NArw']

______

processing column:  SibSp
    root category:  nmbr

 returned columns:
['SibSp_nmbr', 'SibSp_NArw']

______

processing column:  Parch
    root category:  nmbr

 returned columns:
['Parch_nmbr', 'Parch_NArw']

______

processing column:  Ticket
    root category:  hash

 returned columns:
['Tick

In [14]:
test.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Pclass_NArw,Pclass_1.0,Pclass_2.0,Pclass_3.0,Name_NArw,...,Cabin_1010_0,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Embarked_NArw,Embarked_1010_0,Embarked_1010_1
0,1,0.371732,-0.481630,-0.493483,-0.480821,0,0,0,1,0,...,0,1,1,1,0,0,0,0,0,1
1,0,1.308206,0.445182,-0.493483,-0.496660,0,0,0,1,0,...,0,1,1,1,0,0,0,0,1,0
2,1,2.431976,-0.481630,-0.493483,-0.445326,0,0,1,0,0,...,0,1,1,1,0,0,0,0,0,1
3,1,-0.190153,-0.481630,-0.493483,-0.464904,0,0,0,1,0,...,0,1,1,1,0,0,0,0,1,0
4,0,-0.564743,0.445182,0.705697,-0.395663,0,0,0,1,0,...,0,1,1,1,0,0,0,0,1,0


# Custom transformations

Automunge has a library of transformations (documented in the read me). In general, each of these transformations is fit to properties of the train set to enable processing on a consistent basis of additional data.

Each transformation in the libary has a distinct 4 character string identifier, generally aligned with the suffix appender on the returned set. 

We can designate our assignments in the assigncat parameter as follows:

In [15]:
#here we designate min-max scaling to the column 'Fare'
assigncat = {'mnmx':['Fare']}

train, trainID, labels, \
validation1, validationID1, validationlabels1, \
validation2, validationID2, validationlabels2, \
test, testID, testlabels, \
labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
featureimportance, postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assigncat = assigncat,
               printstatus = False)

To view the columns returned from a specific input column can use the column map stored in the postprocess_dict.

In [16]:
train[postprocess_dict['column_map']['Fare']].head()

,Fare_mnmx
709,0.029758
439,0.020495
840,0.015469
720,0.064412
39,0.021942


# Missing data infill

We noted earlier that the MLinfill parameter activates an autoML method for missing data inputation. Let's take a look at this in action. Here we'll turn on ML infill as well as markers for entries subject to infill with the NArw_marker parameter.

In [17]:
train, trainID, labels, \
validation1, validationID1, validationlabels1, \
validation2, validationID2, validationlabels2, \
test, testID, testlabels, \
labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
featureimportance, postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               MLinfill = True,
               NArw_marker=True,
               printstatus = False)

By inspection, if appears that one of the entries in the Age column was subject to infill:

In [18]:
train[postprocess_dict['column_map']['Age']].head()

,Age_nmbr,Age_NArw
709,-1.692485,1
439,0.100052,0
840,-0.745970,0
720,-1.822726,0
39,-1.207437,0


It appears the ML infill is assuming for first row's inputation that this is a very young passenger (remember this is normalized data is reason for the negative value).

Note that the trained models for each feature are saved in the postprocess_dict to enable a consistent inputation basis for subsequent data.

ML infill isn't the only inputation option. Other options like mode, adjacent cell, 0/1, mean, etc can be designated to distinct columns with the assigninfill parameter.

Here we'll demonstrate applying a few different approaches to different columns.

In [19]:
assigninfill = {'MLinfill'  : ['Pclass'],
                'adjinfill' : ['Age'],
                'modeinfill': ['Fare']}

train, trainID, labels, \
validation1, validationID1, validationlabels1, \
validation2, validationID2, validationlabels2, \
test, testID, testlabels, \
labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
featureimportance, postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assigninfill = assigninfill,
               printstatus = False)

In [20]:
train.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Cabin_hash,Pclass_1.0,Pclass_2.0,Pclass_3.0,Name_hash_0,...,Name_hash_9,Name_hash_10,Name_hash_11,Name_hash_12,Name_hash_13,Ticket_hash_0,Ticket_hash_1,Ticket_hash_2,Embarked_1010_0,Embarked_1010_1
709,1,-0.592148,0.432550,0.767199,-0.341261,175,0,0,1,124,...,0,0,0,0,0,541,0,0,0,0
439,1,0.100052,-0.474279,-0.473408,-0.436762,175,0,1,0,369,...,0,0,0,0,0,857,314,0,1,0
840,1,-0.745970,-0.474279,-0.473408,-0.488580,175,0,0,1,315,...,0,0,0,0,0,221,670,0,1,0
720,0,-1.822726,-0.474279,0.767199,0.016014,175,0,1,0,1015,...,0,0,0,0,0,920,0,0,1,0
39,0,-1.207437,0.432550,-0.473408,-0.421837,175,0,0,1,635,...,0,0,0,0,0,578,0,0,0,0


# Conclusion

In closing, as an explanation, the whole point of conducting all of the transformations in a single function is that this application serves to populate a dictionary (the "postprocess_dict") fit to properties of the train data, capturing all of the steps and parameters of transformations, potentially including methods for ML derived missing data inputation, dimensionality reductions, and other various encodings available in the library. This returned dictionary can then be passed to the postmunge(.) function with subsequent data for fully consistent processing on the train set basis.

In [21]:
test, testID, testlabels, \
labelsencoding_dict, postreports_dict \
= am.postmunge(postprocess_dict, df_test)

_______________
Begin Postmunge processing

______

processing column:  Pclass
    root category:  text

 returned columns:
['Pclass_1.0', 'Pclass_2.0', 'Pclass_3.0']

______

processing column:  Name
    root category:  hash

 returned columns:
['Name_hash_0', 'Name_hash_1', 'Name_hash_2', 'Name_hash_3', 'Name_hash_4', 'Name_hash_5', 'Name_hash_6', 'Name_hash_7', 'Name_hash_8', 'Name_hash_9', 'Name_hash_10', 'Name_hash_11', 'Name_hash_12', 'Name_hash_13']

______

processing column:  Sex
    root category:  bnry

 returned columns:
['Sex_bnry']

______

processing column:  Age
    root category:  nmbr

 returned columns:
['Age_nmbr']

______

processing column:  SibSp
    root category:  nmbr

 returned columns:
['SibSp_nmbr']

______

processing column:  Parch
    root category:  nmbr

 returned columns:
['Parch_nmbr']

______

processing column:  Ticket
    root category:  hash

 returned columns:
['Ticket_hash_0', 'Ticket_hash_1', 'Ticket_hash_2']

______

processing column:  Fare


In [22]:
test.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Cabin_hash,Pclass_1.0,Pclass_2.0,Pclass_3.0,Name_hash_0,...,Name_hash_9,Name_hash_10,Name_hash_11,Name_hash_12,Name_hash_13,Ticket_hash_0,Ticket_hash_1,Ticket_hash_2,Embarked_1010_0,Embarked_1010_1
0,1,0.369241,-0.474279,-0.473408,-0.490508,175,0,0,1,276,...,0,0,0,0,0,423,0,0,0,1
1,0,1.330631,0.432550,-0.473408,-0.507194,175,0,0,1,754,...,0,0,0,0,0,804,0,0,1,0
2,1,2.484298,-0.474279,-0.473408,-0.453112,175,0,1,0,382,...,0,0,0,0,0,931,0,0,0,1
3,1,-0.207592,-0.474279,-0.473408,-0.473739,175,0,0,1,271,...,0,0,0,0,0,644,0,0,1,0
4,0,-0.592148,0.432550,0.767199,-0.400792,175,0,0,1,546,...,0,0,0,0,0,75,0,0,1,0
